In [ ]:
import torch
import numpy as np
from torch.distributions import multinomial
from d2l import torch as d2l

In [ ]:
%matplotlib inline
import random
import torch
from d2l import torch as d2l

In [ ]:
def make_data(w, b, num):
    X = torch.normal(0, 1, (num, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 1, y.shape)
    return X, y.reshape(-1,1)

In [ ]:
true_w = torch.tensor([2.0, -3.4])
true_b = 4.2
features, label = make_data(true_w, true_b, 1000)

In [ ]:
print('features:', features[0], '\nlabel:', label[0])

In [ ]:
d2l.set_figsize()
d2l.plt.scatter(features[:, (1)].detach().numpy(), label.detach().numpy(), 1)

In [ ]:
def data_iter(batch_size, features, lebal):
    num = len(features)
    indices = list(range(num))
    random.shuffle(indices)
    for i in range(0, num, batch_size):
        bat = torch.tensor(indices[i: min(i + batch_size, num)])
        yield features[bat], label[bat]

In [ ]:
batch_size = 4

for X, y in data_iter(batch_size, features, label):
    print(X, '\n', y)
    break

In [ ]:
w = torch.normal(0, 0.01, size=(2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [ ]:
def linreg(X, w, b):
    return torch.matmul(X, w) + b

def squared_loss(y_hat, y):
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2

def sgd(params, lr, batch_size):
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch_size
            param.grad.zero_()

In [ ]:
lr = 0.01
epochs = 3
net = linreg
loss = squared_loss

for epoch in range(epochs):
    for X, y in data_iter(batch_size, features, label):
        l = loss(net(X, w, b), y)
        l.sum().backward()
        sgd([w, b], lr, batch_size)
    with torch.no_grad():
        train_l = loss(net(features, w, b), label)
        print(f'epoch {epoch + 1}, loss {float(train_l.mean()):f}')

In [ ]:
w,b

In [ ]:
print(f'w的估计误差: {true_w - w.reshape(true_w.shape)}')
print(f'b的估计误差: {true_b - b}')

## 线性回归欧姆定律 U=IR

In [ ]:
R = torch.tensor([5]).reshape(1,1)
R

In [ ]:
I = torch.arange(0,1,0.01).reshape(-1,1) # n*1
true_R = 2 #1*1
U = I * true_R
U += torch.normal(0, 0.01, U.shape)

In [ ]:
d2l.plt.scatter(I[:, (0)].detach().numpy(), U.detach().numpy(), 1)

In [ ]:
r = torch.tensor(3.0, requires_grad=True)
r

In [ ]:
def Ohm(I, R):
    return I * R

In [ ]:

u = Ohm(I,r)
l = loss(u, U)
l.sum().backward()
# r.grad.zero_()
r.grad

In [ ]:
lr = 0.003
epochs = 3
batch_size = 4
net = Ohm
loss = squared_loss

for epoch in range(epochs):
    for X, y in data_iter(batch_size, I, U):
        l = loss(net(I, r), U)
        l.sum().backward()
        sgd([r], lr, batch_size)
    with torch.no_grad():
        train_l = loss(net(I, r), U)
        print(f'epoch {epoch + 1}, loss {float(train_l.mean()):f}')

In [ ]:
r

## 线性回归的简洁实现

In [147]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

In [148]:
def load_array(data_array, batch_size, is_train=True):
    dataset = data.TensorDataset(*data_array)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [149]:
next(iter(data_iter))

[tensor([[ 0.1758,  0.0564],
         [-0.6012,  1.4644],
         [ 1.0681,  1.8295],
         [ 0.7229, -1.3322],
         [ 0.3579, -0.5269],
         [-0.7895,  0.1846],
         [-0.2601,  1.8263],
         [-1.1942,  0.2848],
         [ 0.5861,  1.2901],
         [-0.4604,  0.3692]]),
 tensor([[ 4.3793],
         [-1.9926],
         [ 0.1032],
         [10.1760],
         [ 6.6868],
         [ 1.9843],
         [-2.5270],
         [ 0.8330],
         [ 0.9873],
         [ 2.0053]])]

In [150]:
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

loss = nn.MSELoss()

In [151]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)
trainer

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)

In [153]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000093
epoch 2, loss 0.000093
epoch 3, loss 0.000094


In [155]:
w = net[0].weight.data
b = net[0].bias.data
w, b

(tensor([[ 2.0002, -3.3989]]), tensor([4.2001]))